In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


2024-08-11 06:57:03.896000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 06:57:03.896090: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 06:57:03.897950: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def fetch(path):
    file=open(path,"r")
    content1=file.read()
    file.close()
    content1=(content1.split("\n"))
    text=[]
    label=[]
    try:
        for j,i in enumerate(content1):
            text1,label1=i.split(";")
            text.append(text1)
            label.append(label1)
    except Exception as e:
        print("")
    return pd.DataFrame({'text': text, 'label': label})

In [3]:
train_data=fetch("/kaggle/input/emotions-dataset-for-nlp/train.txt")

In [4]:
train_data = train_data[train_data['label'] != "love"]

In [5]:
train_data

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
4,i am feeling grouchy,anger
5,ive been feeling a little burdened lately wasn...,sadness
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [6]:
train_data2=pd.read_csv("/kaggle/input/go-emotions-google-emotions-dataset/go_emotions_dataset.csv")

In [7]:
set(train_data["label"])

{'anger', 'fear', 'joy', 'sadness', 'surprise'}

In [8]:
train_data2=train_data2[["text","disgust","neutral"]]

In [9]:
train_data2=train_data2[~((train_data2["disgust"]==0) & (train_data2["neutral"]==0))]

In [10]:
train_data2

,text,disgust,neutral
2,"You do right, if you don't care then fuck 'em!",0,1
4,"[NAME] was nowhere near them, he was by the Fa...",0,1
10,"I have, and now that you mention it, I think t...",0,1
12,BUT IT'S HER TURN! /s,0,1
13,That is odd.,1,0
...,...,...,...
211209,OH YEAH!!,0,1
211210,Let me give you a hint: THEY PLAY IN BOSTON!!!,0,1
211211,to google cuz I wasn't alive back then but dam...,0,1
211213,"Wow, she headlines two shows now",0,1


In [11]:
val_data=fetch("/kaggle/input/emotions-dataset-for-nlp/val.txt")

In [12]:
val_data= val_data[val_data['label'] != "love"]

In [13]:
test_data=fetch("/kaggle/input/emotions-dataset-for-nlp/test.txt")

In [14]:
test_data= test_data[test_data['label'] != "love"]

In [ ]:
pip install --upgrade nltk


In [15]:
!pip install nltk

In [16]:
import nltk

# Specify the directory for NLTK data
# nltk.data.path.append('/usr/share/nltk_data')

# Download necessary NLTK data
nltk.download("wordnet")
nltk.download('stopwords')
nltk.download('punkt_tab')

# Import necessary modules
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd

# Initialize the lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define a preprocessing function
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_tokens)

# Apply preprocessing to both columns
train_data["text"] = train_data['text'].apply(preprocess_text)
test_data['text'] = test_data['text'].apply(preprocess_text)
val_data['text'] = val_data['text'].apply(preprocess_text)

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [17]:
train_data2['text'] = train_data2['text'].apply(preprocess_text)

In [18]:
train_data2

,text,disgust,neutral
2,right care fuck,0,1
4,name nowhere near falcon,0,1
10,mention think triggered nostalgia,0,1
12,turn,0,1
13,odd,1,0
...,...,...,...
211209,oh yeah,0,1
211210,let give hint play boston,0,1
211211,google cuz alive back damn like em,0,1
211213,wow headline two show,0,1


In [ ]:
def fetch_embedding(path):
    file = open(path, 'r', encoding = 'utf8')
    content = file.readlines()
    file.close()
    embeddings = {}
    for line in content:
        line = line.split()
        embeddings[line[0]] = np.array(line[1:], dtype = float)
    return embeddings


In [ ]:
import numpy as np
embeddings=fetch_embedding("/kaggle/input/asdfghjk/glove.6B.100d.txt")

In [19]:
def get_maxlen(data):
    maxlen = 0
    for i in range(len(data)):
        maxlen = max(maxlen, len(data[i]))
    return maxlen

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data["text"])
tokenizer.fit_on_texts(test_data["text"])
tokenizer.fit_on_texts(val_data["text"])
tokenizer.fit_on_texts(train_data2["text"])
word2index = tokenizer.word_index

In [ ]:
len(word2index)

In [21]:
train_tokens = tokenizer.texts_to_sequences(train_data["text"])
test_tokens = tokenizer.texts_to_sequences(test_data["text"])
val_tokens = tokenizer.texts_to_sequences(val_data["text"])
train2_tokens = tokenizer.texts_to_sequences(train_data2["text"])

# print(Xtokens)
# print("========================================")
maxlen_train = get_maxlen(train_tokens)
maxlen_test = get_maxlen(test_tokens)
maxlen_val=get_maxlen(val_tokens)
maxlen_train2=get_maxlen(train2_tokens)

In [23]:
import json
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w') as json_file:
    json.dump(tokenizer_json, json_file)

In [ ]:
maxlen=max(maxlen_val,maxlen_train,maxlen_test,maxlen_train2)

In [ ]:
print(maxlen_val,maxlen_train,maxlen_test,maxlen_train2)
print(maxlen)

In [ ]:
train_data2['new_column'] = train_data2.apply(lambda row: 1 if row['disgust'] == 1 else (4 if row['neutral'] == 1 else 0), axis=1)

# Remove the original columns
train_data2.drop(columns=['disgust', 'neutral'], inplace=True)

In [ ]:
train_data2

In [ ]:
Y_train=train_data["label"]
Y_test=test_data["label"]
Y_val=val_data["label"]
Y_train2=train_data2["new_column"]

In [ ]:
set(Y_train)

In [ ]:
mapping={ "anger":0,  "disgust":1,  "fear":2, "joy":3,  "neutral":4,  "sadness":5,  "surprise":6}
Y_train = [mapping[label] for label in Y_train]
Y_test = [mapping[label] for label in Y_test]
Y_val = [mapping[label] for label in Y_val]

In [ ]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
Y_val = to_categorical(Y_val)
Y_train2=to_categorical(Y_train2)

In [ ]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    if(word in embeddings.keys()):
        embed_vector = embeddings[word]
        embedding_matrix[i] = embed_vector

In [ ]:
embedding_matrix.shape

In [ ]:
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# instantiate a distribution strategy
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
train_tokens

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define TPU strategy if needed
X_train = train_tokens
X_val = val_tokens
X_test=test_tokens
X_train2=train2_tokens
maxlen = max(max(len(seq) for seq in X_train), 
             max(len(seq) for seq in X_test), 
             max(len(seq) for seq in X_val),
             max(len(seq) for seq in X_train2))

# Pad sequences to ensure uniform length
X_train_padded = pad_sequences(X_train, maxlen=maxlen, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test, maxlen=maxlen, padding='post', truncating='post')
X_val_padded = pad_sequences(X_val, maxlen=maxlen, padding='post', truncating='post')
X_train2_padded = pad_sequences(X_train2, maxlen=maxlen, padding='post', truncating='post')

# Convert to NumPy arrays
X_train = np.array(X_train_padded)
X_val = np.array(X_val_padded)
X_test = np.array(X_test_padded)
X_train2 = np.array(X_train2_padded)
# Assuming Ytrain is already a NumPy array
y_train = np.array(Y_train)
y_test = np.array(Y_test)
y_val = np.array(Y_val)
y_train2 = np.array(Y_train2)



y_train = np.argmax(Y_train, axis=-1)
y_val = np.argmax(Y_val, axis=-1)
y_test = np.argmax(Y_test, axis=-1)
y_train2 = np.argmax(Y_train2, axis=-1)

y_train_one_hot = to_categorical(y_train, num_classes=7)
y_train2_one_hot = to_categorical(y_train2, num_classes=7)
y_val_one_hot = to_categorical(y_val, num_classes=7)
y_test_one_hot = to_categorical(y_test, num_classes=7)

# Update the model compilation and training
# Define Parameters
embed_size = 100
# embedding_matrix = np.random.rand(len(word2index) + 1, embed_size)  # Define this properly if needed

# Define Title Neural Network
with tpu_strategy.scope():
    inputs = Input(shape=(maxlen,))
    x = Embedding(input_dim=len(word2index) + 1,
                  output_dim=embed_size,
                  weights=[embedding_matrix],
                  input_length=maxlen,
                  trainable=False)(inputs)
    x = SpatialDropout1D(0.2)(x)
    x = LSTM(100, dropout=0.2, recurrent_dropout=0.2)(x)
    x = Dense(50, activation='relu')(x)
    outputs = Dense(7, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()



In [ ]:
X_train2.shape,y_train2.shape

In [ ]:
X_train.shape,y_train.shape

In [ ]:
train = np.vstack((X_train, X_train2)) 

In [ ]:
train.shape

In [ ]:
yy_train=np.vstack((y_train_one_hot,y_train2_one_hot))

In [ ]:
yy_train.shape

In [ ]:

history = model.fit(
    train,  # Training data
    yy_train,  # Training labels (one-hot encoded)
    epochs=50,  # Number of epochs
    batch_size=32,  # Batch size
    validation_data=(X_val, y_val_one_hot),  # Use a validation split (one-hot encoded)
    verbose=1  # Verbosity mode
)

In [ ]:
y_train_one_hot.shape

In [ ]:
with tpu_strategy.scope():
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='model_for_text_emotion_updated(1).keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
    history = model.fit(
        train,  # Training data
        yy_train,  # Training labels (one-hot encoded)
        epochs=1,  # Number of epochs
        batch_size=32,  # Batch size
        validation_data=(X_val, y_val_one_hot),  # Use a validation split (one-hot encoded)
        verbose=1,  # Verbosity mode
        callbacks=[checkpoint_callback]
    )

In [ ]:
with tpu_strategy.scope():
    model.save("model_arch_for_text_emotion_updated.h5")

In [ ]:
X_train = train_tokens
X_val = val_tokens
X_test=test_tokens
maxlen = max(max(len(seq) for seq in X_train), 
             max(len(seq) for seq in X_test), 
             max(len(seq) for seq in X_val))

# Pad sequences to ensure uniform length
X_train_padded = pad_sequences(X_train, maxlen=maxlen, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test, maxlen=maxlen, padding='post', truncating='post')
X_val_padded = pad_sequences(X_val, maxlen=maxlen, padding='post', truncating='post')

# Convert to NumPy arrays
X_train = np.array(X_train_padded)
X_val = np.array(X_val_padded)
X_test = np.array(X_test_padded)

# Assuming Ytrain is already a NumPy array
y_train = np.array(Y_train)
y_test = np.array(Y_test)
y_val = np.array(Y_val)



y_train = np.argmax(Y_train, axis=-1)
y_val = np.argmax(Y_val, axis=-1)
y_test = np.argmax(Y_test, axis=-1)


y_train_one_hot = to_categorical(y_train, num_classes=6)
y_val_one_hot = to_categorical(y_val, num_classes=6)
y_test_one_hot = to_categorical(y_test, num_classes=6)

In [ ]:
X_test[0]

In [ ]:
with open('/kaggle/working/model_architecture_for_text_emotion_updated_json.json', 'w') as json_file:
    json_file.write(model.to_json())

In [ ]:
from tensorflow.keras.models import load_model
# model=load_model('/kaggle/input/zxcvbjdsx/model_arch_for_text_emotion.h5')
model.load_weights('/kaggle/input/zxcvbjdsx/model_for_text_emotion_130.keras')
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot, verbose=1)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')